In [1]:
import sys
sys.path.append('../../../GraphGallery/')
sys.path.append('../../../GraphAdv/')

import graphgallery
from graphgallery.nn.models import DenseGCN
from graphadv.attack.untargeted import PGD


import matplotlib.pyplot as plt

In [2]:
from graphgallery.data import NPZDataset, Planetoid
data = Planetoid('cora', root="~/GraphData/datasets/", verbose=False)
adj, x, labels = data.graph.unpack()
idx_train, idx_val, idx_test = data.split()


Downloaded dataset files have existed.
Processing...
Processing completed.


# Before Attack

In [3]:
x_norm = graphgallery.normalize_x(x, norm='l1')

In [4]:
surrogate = DenseGCN(adj, x_norm, labels, device='GPU', norm_x=None, seed=123)
surrogate.build(32)
his_before = surrogate.train(idx_train, idx_val, verbose=1, epochs=100)
loss, accuracy = surrogate.test(idx_test)
print(f'Test loss {loss:.5}, Test accuracy {accuracy:.2%}')

<Loss = 0.7786 Acc = 0.9714 Val_Loss = 1.2553 Val_Acc = 0.7900 >: 100%|██████████| 100/100 [00:01<00:00, 70.65it/s]


Test loss 1.5142, Test accuracy 82.40%


# White-box attack 

In [5]:
attacker = PGD(adj, x_norm, labels, idx_train, idx_unlabeled=idx_test, surrogate=surrogate, device="GPU", seed=None)
attacker.reset()
attacker.attack(0.05, structure_attack=True, feature_attack=False)
# show logs
attacker.show_edge_flips(detail=False)

Random Sampling: 100%|██████████| 20/20 [00:00<00:00, 153.99it/s]


Flip 261 edges, 261 added, 0 removed. Added edges with different classes: 98.08%, removed edges with the same classes: 0.00%


# After Attack

In [6]:
# NOTE: after calling `preprocess`, the adjacency matrix and attribute matrix of surrogate model
# are registered as the perturbed ones
# DO NOT use this surrogate model again for surrogate attack
# if you really do, please call `surrogate.preprocess(adj, x_norm)` first.
surrogate.preprocess(attacker.A, x_norm)
loss, accuracy = surrogate.test(idx_test)
print(f'Test loss {loss:.5}, Test accuracy {accuracy:.2%}')

Test loss 1.6143, Test accuracy 74.40%
